In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

import tweepy
from kafka import KafkaProducer
from tweepy import OAuthHandler
from tweepy.streaming import StreamListener
from tweepy import Stream
import json

# Declare a Python class (MyListener) that inherit from tweepy.StreamListener
# Then we override the on_status method that will be automatically used once
# Twitter "sends" us a new tweet.
class MyListener(StreamListener):
    
  def __init__(self, producer, producer_topic):
    super().__init__()
    self.producer = producer
    self.producer_topic = producer_topic

  def on_status(self, status):
    tweet = {"tweet_id": status.id, "user_screen_name": status.user.screen_name,
             'text': status.text,
             "time": status.created_at.strftime("%Y-%m-%d %H:%M:%S"), "tweet_geo": status.geo,
             "tweet_retweet": status.retweet_count, "tweet_favourite": status.favorite_count,
             "tweet_device": str(status.source).replace(";", ",,"),
             "user_description": str(status.user.description).replace(";", ",,"),
             "user_follower": status.user.followers_count, "user_verified": status.user.verified,
             'hashtags': []
            }
    if status.entities.get('hashtags') is not None:
        hashtags = status.entities.get('hashtags')
        tweet['hashtags'] = [ h.get('text') for h in hashtags ]
        self.producer.send(topic=self.producer_topic, value=tweet)
    

# Set consumer key, consumer secret, access token and access token secret
# Read them from your Twitter dev page
consumer_key = "mM0U7Sxn3a3VayLXuABffIQKo"
consumer_secret = "XoL7Y93rrtxgu7cZsooZE4d116O0L6sTIT0bvRbPswemtur37n"
access_token = "1329171042225545217-DmwPJj3GLrAnqtO8j5rvNJxuqD4M7V"
access_token_secret = "XfTCpPpzGuO1wirwgXosHR2ydh7NMG5ypZJjzmuii24KF"

# Authenticate with the OAuth protocol
# See: https://en.wikipedia.org/wiki/OAuth
auth = OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)

# Get the API object passing the authentication object
# See http://docs.tweepy.org/en/latest/api.html to discover the available methods
api = tweepy.API(auth)

producer = KafkaProducer(
  bootstrap_servers=["kafka:9092"],
  value_serializer=lambda v: json.dumps(v).encode("utf-8"))

listener = MyListener(producer=producer, producer_topic="test_json")
# We need an api to stream, so we reuse the api object we create before.
# Thus, we create a Stream object.
stream = Stream(auth = api.auth, listener=listener)

# A number of twitter streams are available through Tweepy.
# For more information on the capabilities and limitations of the different
# streams see Twitter Streaming API Documentation.

# In this example we will use filter to stream all tweets containing the word "Milano".
# The track parameter is an array of search terms to stream.
stream.filter(track=['Vikings', 'Lagertha', 'Ragnar', 'HistoryVikings', "Jupiter's Legacy", 'Locke and key',
                    'Peaky Blinders', 'Sex Education 2', 'Casa de papel', 'Sex Education 3',
                    'Stranger Things', 'The last dance', 'The OA', 'Narcos', 'Ragnarok', 
                    'dark netflix', 'dark jonas', 'dark martha',
                    'Ozark', 'Martin Byrde', 'Wendy Byrde', 'Bruce Liddell', 
                    'Lucifer', 'Chloe Decker', 'Daniel Espinoza', 'Lucifer Morningstar', 'Amenadiel',
                    'Elite 3 netflix', 'Elite 4 netflix', 'The Witcher', 'kissing booth 3', 'kissing booth 2',
                    'The Umbrella Academy', 'Vanya Hargreeves', 'Luther Hargreeves', 'Diego Hargreeves', 'Allison Hargreeves',
                    'Klaus Hargreeves', 'Cinque Hargreeves', 'Pogo', 'Reginald Hargreeves',
                    'geralt of rivia', 'Suburra',
                    'Breaking Bad',
                     'The Walking Dead',
                     'Sherlock',
                     'Dexter',
                     'House of Cards',
                     'Black Mirror',
                     'Arrow',
                     'Supernatural',
                     'Suits',
                     'Better Call Saul',
                     'Homeland',
                     'The Flash',
                     'The Vampire Diaries',
                     'Orange Is the New Black',
                     'American Horror Story',
                     'Arrested Development',
                     '13 Reasons Why',
                     "Grey's Anatomy",
                     'Avatar The Last Airbender',
                     'Hannibal',
                     'Parks and Recreation',
                     'Gotham',
                     'Once Upon a Time',
                     'Mad Men',
                     'The Blacklist',
                     'Twin Peaks',
                     'Person of Interest',
                     'Criminal Minds',
                     'Gossip Girl',
                     "That '70s Show",
                     'The Crown',
                     'The End of the F***ing World',
                     'Altered Carbon',
                     'Sense8',
                     'Glee',
                     'How to Get Away with Murder',
                     'Riverdale',
                     'Supergirl',
                     'Cobra Kai',
                     'Love Death & Robots',
                     'Designated Survivor',
                     'Merlin',
                     'Star Trek',
                     'Lost in Space',
                     'Naruto'], languages=["en"])